In [69]:
import numpy as np
import pandas as pd

In [227]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
Id = test["Id"]

In [228]:
num_c = list(train.select_dtypes(include=['int64','float64']).columns)[:-1]

In [229]:
cat_c = list(train.select_dtypes(include=['object']).columns)

In [230]:
target_c = "SalePrice"

In [231]:
train.drop("Id", axis = 1, inplace = True)
train = pd.get_dummies(train, columns = cat_c)
test.drop("Id", axis = 1, inplace = True)
test = pd.get_dummies(test, columns = cat_c)

In [232]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [233]:
cat_c_new = []
for i in cat_c:
  cat_c_new.extend(filter(lambda x: x.startswith(i), test.columns))
cat_c = cat_c_new

In [237]:
cols = []
for i in test.columns:
  if i != target_c:
    cols.append(i)

In [238]:
from sklearn.model_selection import train_test_split
X_tr, X_t, y_tr, y_t = train_test_split(train[cols], train[target_c], test_size = 0.2)

In [239]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler().fit(X_tr)
X_tr = std_scaler.transform(X_tr)
X_t = std_scaler.transform(X_t)

In [240]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_tr, y_tr)

In [241]:
from sklearn import metrics

In [242]:
def score(test, pred):
  print("MSE: " + str(metrics.mean_squared_error(test, pred)))
  print("R^2: " + str(metrics.r2_score(test, pred)))

In [243]:
score(y_t, lr.predict(X_t))

MSE: 844593314.0162903
R^2: 0.8561449562167324


In [244]:
from sklearn.ensemble import GradientBoostingRegressor

In [245]:
gr = GradientBoostingRegressor().fit(X_tr, y_tr)

In [246]:
score(y_t, gr.predict(X_t))

MSE: 484507089.3372305
R^2: 0.9174765092343999


In [259]:
test_x = std_scaler.transform(test)
prediction = pd.DataFrame([Id,  lr.predict(test_x)]).transpose()
prediction["Id"] = prediction["Id"].astype("int32")
prediction.set_index("Id", inplace = True)
prediction.columns=["SalePrice"]
prediction.to_csv("SalePrice.csv")